In [2]:
# pypdf
# langchain
# torch
# accelerate
# bitsandbytes
# transformers
# sentence_transformers
# faiss_gpu

!pip install -r /content/requirement.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 28.2 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.15
    Uninstalling langchain-core-0.3.15:
      Successfully uninstalled langchain-core-0.3.15


In [5]:
!pip install pypdf
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.5 MB/s eta 0:00:00


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

DATA_PATH = "/content/DATA"
DB_FAISS_PATH = "vectorstores/DB_FAISS"

def create_vector_db():
    loader = DirectoryLoader(DATA_PATH,glob = '*.pdf',loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=50)
    texts = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={"device":"cuda"})

    db = FAISS.from_documents(texts,embeddings)
    db.save_local(DB_FAISS_PATH)

if __name__ == "__main__":
    create_vector_db()

<ipython-input-7-4180cbf1c76c>:15: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={"device":"cuda"})
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
pip install ctransformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 50.7 MB/s eta 0:00:00


In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA


DB_FAISS_PATH = "/content/vectorstores/DB_FAISS"

custom_prompt_template = """
Use the following pieces of information to answer the user question.

Context: {context}
Question: {question}

Only returns the helpful answer below and nothing else.
Helpful Answer:

"""
def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vector stores

    """
    prompt = PromptTemplate(template = custom_prompt_template,input_variables = ["context","question"])
    return prompt

def load_llm():
    """
    Load the LLM model for QA retrieval

    """
    # llm = CTransformers(model="/content/drive/MyDrive/DoColab/llama-2-7b-chat.ggmlv3.q8_0.bin",
    # model_type = "llama",
    # max_new_tokens = 512,
    # temperature = 0.5)

    llm = CTransformers(
        model="/content/drive/MyDrive/llama-2-7b-chat.ggmlv3.q4_0.bin",
        model_type="llama",
        max_new_tokens=512,
        temperature=0.5,
        quantization="int4"
    )

    return llm

def retrieval_qa_chain(llm,prompt,db):
    qa_chain = RetrievalQA.from_chain_type(llm = llm,
    chain_type = "stuff",
    retriever = db.as_retriever(search_kwargs = {"k":2}),
    return_source_documents = True,
    chain_type_kwargs = {"prompt":prompt})
    return qa_chain


def qa_bot():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device":"cpu"})

    db = FAISS.load_local(DB_FAISS_PATH,embeddings, allow_dangerous_deserialization=True)
    llm = load_llm()
    qa_prompt = set_custom_prompt()
    qa = retrieval_qa_chain(llm,qa_prompt,db)
    return qa

def final_result(query):
    qa_result = qa_bot()
    response = qa_result({'query':query})
    return response

In [10]:
response = final_result("What are the problems?")
print(response)

<ipython-input-9-15d4dc002683>:69: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_result({'query':query})


{'query': 'What are the problems?', 'result': 'The problems that the proposed solution aims to address are:\n\n1. Lack of timely medical care for crash victims due to ambulances being located far away.\n2. High percentage of road accidents caused by sleep-deprivedent drivers,riveduct drivers (4rivedrive drivers (4rivedned drivers. drivers, which can bearded drivers. drivers (about drivers. drivers. drivers. drivers. drivers. drivers. drivers,erented drivers,riven drivers. drivers. drivers (approved drivers. drivers. drivers. drivers. drivers. drivers. drivers (4rivedrivedent drivers. drivers. drivers,rivedrivedrive drivers. drivers. drivers. drivers,ri...', 'source_documents': [Document(metadata={'source': '/content/DATA/Samskarana_VolksKraft.pdf', 'page': 1}, page_content='MOBILE APPFOR COMPLETETRACKING\nApp Architecture \nSleep-deprived drivers remain responsible for about 40% of the road accidents (the Hindu)\nInjuries due to accidental crash are the 8th leading cause of death world

In [ ]:
!pip install pyngrok

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok

port_no = 3000
app = Flask(__name__)
ngrok.set_auth_token("2lQMOpiWOTnWyFEIFUhWD8OiFJb_7g1rnALXR6nF9EXYBRxqq")

# Disable CORS
app.config['CORS_HEADERS'] = 'Content-Type'

# Define CORS handler
@app.after_request
def after_request(response):
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Headers', 'Content-Type,Authorization')
    response.headers.add('Access-Control-Allow-Methods', 'GET,PUT,POST,DELETE')
    response.headers.add('Access-Control-Allow-Credentials', 'true')
    return response

public_url = ngrok.connect(port_no).public_url

@app.route('/', methods=['POST'])
def post_request():
    try:
        data = request.json
        topic_value = data.get('topic', 'Topic not found')

        if topic_value:
            output_text = final_result(topic_value)

            return jsonify({
                'success': True,
                'message': 'post request works properly',
                'topic': output_text,
            })
        else:
            return jsonify({
                'success': False,
                'message': 'Topic not provided in the request.',
            })
    except Exception as e:
        return jsonify({
            'success': False,
            'message': str(e),
        })

print(f"To access the Global link, please click {public_url}")
app.run(port=port_no)


To access the Global link, please click https://3368-34-125-153-98.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:3000
INFO:werkzeug:Press CTRL+C to quit
